In [1]:
# 1. Instalación de dependencias (ejecutar en Colab)
import importlib.util, subprocess, sys
def ensure_package(package_name):
    if importlib.util.find_spec(package_name) is None:
        print(f"Instalando {package_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
    else:
        print(f"{package_name} ya instalado.")

ensure_package("optuna")
ensure_package("xgboost")
ensure_package("kaggle")
ensure_package("joblib")

print("Dependencias listas.")



Instalando optuna ...
xgboost ya instalado.
kaggle ya instalado.
joblib ya instalado.
Dependencias listas.


In [2]:
# 2. Imports
import os, zipfile
import numpy as np
import pandas as pd
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from joblib import dump

print("Imports cargados.")



Imports cargados.


In [3]:

# 3. Configurar Kaggle y descargar datos

from google.colab import files

from google.colab import files
print("Sube tu kaggle.json (API token de Kaggle) cuando se abra el diálogo...")
uploaded = files.upload()

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
for fname in uploaded.keys():
    if fname == "kaggle.json":
        dest = os.path.join(os.path.expanduser("~/.kaggle"), "kaggle.json")
        os.replace(fname, dest)
        os.chmod(dest, 0o600)
        print("kaggle.json copiado a ~/.kaggle")
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -p /content
!unzip -o /content/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -d /content/udea_saber_pro

print("Descarga y extracción completadas.")


Sube tu kaggle.json (API token de Kaggle) cuando se abra el diálogo...


Saving kaggle.json to kaggle.json
kaggle.json copiado a ~/.kaggle
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.69GB/s]
Archive:  /content/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: /content/udea_saber_pro/submission_example.csv  
  inflating: /content/udea_saber_pro/test.csv  
  inflating: /content/udea_saber_pro/train.csv  
Descarga y extracción completadas.


In [4]:

# 4. Cargar train y test

TRAIN_PATH = "/content/udea_saber_pro/train.csv"
TEST_PATH  = "/content/udea_saber_pro/test.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

print("Train shape:", train.shape)
print("Test shape:", test.shape)

display(train.head())
display(test.head())



Train shape: (692500, 21)
Test shape: (296786, 20)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [5]:
# 5. Función reusable de imputación (media / moda)
def imputar_media_moda(df, id_col=None, target_col=None):
    """
    Imputa media en numéricas y moda en categóricas (ignora id y target)
    Retorna: df_imputado, columnas_numericas, columnas_categoricas
    """
    df_imputado = df.copy()
    excluir = set()
    if id_col and id_col in df_imputado.columns:
        excluir.add(id_col)
    if target_col and target_col in df_imputado.columns:
        excluir.add(target_col)
    columnas_numericas = df_imputado.select_dtypes(include=[np.number]).columns.tolist()
    columnas_categoricas = df_imputado.select_dtypes(include=['object','category','bool']).columns.tolist()
    columnas_numericas = [c for c in columnas_numericas if c not in excluir]
    columnas_categoricas = [c for c in columnas_categoricas if c not in excluir]
    for col in columnas_numericas:
        if df_imputado[col].isna().any():
            df_imputado[col] = df_imputado[col].fillna(df_imputado[col].mean())
    for col in columnas_categoricas:
        if df_imputado[col].isna().any():
            moda = df_imputado[col].mode(dropna=True)
            if len(moda) > 0:
                df_imputado[col] = df_imputado[col].fillna(moda.iloc[0])
    return df_imputado, columnas_numericas, columnas_categoricas

print("Función de imputación definida.")


Función de imputación definida.


In [6]:
# 6. Definir preprocesador con ColumnTransformer
ID_COL = "ID"
TARGET_COL = "RENDIMIENTO_GLOBAL"

train_imp, num_cols, cat_cols = imputar_media_moda(train, id_col=ID_COL, target_col=TARGET_COL)
feature_cols = [c for c in train.columns if c not in [ID_COL, TARGET_COL]]
numeric_features = [c for c in feature_cols if c in num_cols]
categorical_features = [c for c in feature_cols if c in cat_cols]

numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="mean"))])
categorical_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),
                                          ("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

print("Preprocesador definido.")
print("num features:", len(numeric_features), "cat features:", len(categorical_features))



Preprocesador definido.
num features: 5 cat features: 14


In [7]:
# 7. Definir X, y, aplicar label mapping y split
X = train[feature_cols].copy()
y = train[TARGET_COL].copy()

# Mapeo obligatorio: texto -> números (XGBoost necesita labels numéricos en multiclass)
label_map = {
    "bajo": 0,
    "medio-bajo": 1,
    "medio-alto": 2,
    "alto": 3
}

# Si y es texto/categoría mapeamos; si ya es numérico, no lo tocamos
if y.dtype == object or y.dtype.name == "category":
    y = y.map(label_map)

num_clases = len(set(y.unique()))
print("num_clases:", num_clases)

# Train/valid split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
print("X_train:", X_train.shape, "X_valid:", X_valid.shape)


num_clases: 4
X_train: (484750, 19) X_valid: (207750, 19)


In [8]:
# 8. Función para construir pipelines
def make_base_pipeline(model):
    return Pipeline(steps=[("preprocess", preprocessor), ("model", model)])

print("make_base_pipeline listo.")



make_base_pipeline listo.


In [9]:
# 9. Submuestreo para Optuna (ajusta train_size si quieres más/menos)
X_optuna, _, y_optuna, _ = train_test_split(X_train, y_train, train_size=0.3, stratify=y_train, random_state=42)
print("Subconjunto para Optuna:", X_optuna.shape)



Subconjunto para Optuna: (145425, 19)


In [10]:
# 10. Objetivos de Optuna para XGB, SVM

import warnings
from sklearn.exceptions import FitFailedWarning

def objective_xgb(trial):
    model = XGBClassifier(
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        max_depth=trial.suggest_int("max_depth", 2, 8),
        n_estimators=trial.suggest_int("n_estimators", 50, 200),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        gamma=trial.suggest_float("gamma", 0.0, 5.0),
        subsample=trial.suggest_float("subsample", 0.5, 1.0),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.5, 1.0),
        reg_alpha=trial.suggest_float("alpha", 0.0, 1.0),
        objective='multi:softprob',
        eval_metric='mlogloss',
        tree_method='hist',
        random_state=42

    )
    pipe = make_base_pipeline(model)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=FitFailedWarning)
        scores = cross_val_score(pipe, X_optuna, y_optuna, cv=cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return scores.mean()


def objective_svm_linear(trial):

    # Hiperparámetros que SÍ tiene SGDClassifier (SVM lineal)
    alpha = trial.suggest_float("alpha", 1e-5, 1e-2, log=True)
    penalty = trial.suggest_categorical("penalty", ["l2", "l1", "elasticnet"])
    max_iter = trial.suggest_int("max_iter", 500, 2000)

    X_sub, _, y_sub, _ = train_test_split(
        X_optuna, y_optuna,
        train_size=0.5,          # usa solo el 50% de X_optuna
        stratify=y_optuna,
        random_state=trial.number
    )

    # Modelo SVM lineal con SGD
    model = SGDClassifier(
        loss="hinge",               # hinge = SVM lineal
        alpha=alpha,
        penalty=penalty,
        max_iter=max_iter,
        class_weight="balanced",
        random_state=42
    )
    pipe = make_base_pipeline(model)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    scores = cross_val_score(
        pipe,
        X_sub,
        y_sub,
        cv=cv,
        scoring="accuracy",
        n_jobs=-1,
        error_score="raise"
    )

    return scores.mean()

print("Objetivos Optuna definidos.")


Objetivos Optuna definidos.


In [11]:
# 11. Ejecutar Optuna (ajusta n_trials según tiempo disponible)
n_trials_xgb = 20
n_trials_svm_lineal = 3


study_xgb = optuna.create_study(direction='maximize', study_name='xgb_saberpro')
study_xgb.optimize(objective_xgb, n_trials=n_trials_xgb)
print("Best XGB:", study_xgb.best_params)

study_svm_linear = optuna.create_study(direction="maximize", study_name="svm_lineal_saberpro")
study_svm_linear.optimize(objective_svm_linear, n_trials_svm_lineal)
print("Best SVM lineal:", study_svm_linear.best_params)


[I 2025-11-26 21:01:53,220] A new study created in memory with name: xgb_saberpro
[I 2025-11-26 21:02:10,108] Trial 0 finished with value: 0.4124325253567131 and parameters: {'learning_rate': 0.020257556188881545, 'max_depth': 8, 'n_estimators': 193, 'min_child_weight': 2, 'gamma': 3.357557873380443, 'subsample': 0.8907580481145366, 'colsample_bytree': 0.6240531306984001, 'alpha': 0.5687906087225041}. Best is trial 0 with value: 0.4124325253567131.
[I 2025-11-26 21:02:14,337] Trial 1 finished with value: 0.38995358432181537 and parameters: {'learning_rate': 0.10022141623557902, 'max_depth': 2, 'n_estimators': 58, 'min_child_weight': 2, 'gamma': 4.882261881791611, 'subsample': 0.7730566581798235, 'colsample_bytree': 0.902127695907327, 'alpha': 0.6757870731227055}. Best is trial 0 with value: 0.4124325253567131.
[I 2025-11-26 21:02:19,471] Trial 2 finished with value: 0.40470345538937597 and parameters: {'learning_rate': 0.0991950427148741, 'max_depth': 4, 'n_estimators': 60, 'min_child_

Best XGB: {'learning_rate': 0.2583555627932424, 'max_depth': 7, 'n_estimators': 158, 'min_child_weight': 8, 'gamma': 2.88501958097666, 'subsample': 0.8145087835651421, 'colsample_bytree': 0.8842064341622737, 'alpha': 0.20484112316281267}


[I 2025-11-26 21:04:40,858] Trial 0 finished with value: 0.3160970308095792 and parameters: {'alpha': 0.008203850788364248, 'penalty': 'elasticnet', 'max_iter': 810}. Best is trial 0 with value: 0.3160970308095792.
[I 2025-11-26 21:04:58,806] Trial 1 finished with value: 0.2543322360729067 and parameters: {'alpha': 0.005882585165661186, 'penalty': 'l2', 'max_iter': 1651}. Best is trial 0 with value: 0.3160970308095792.
[I 2025-11-26 21:05:14,296] Trial 2 finished with value: 0.24782704358378096 and parameters: {'alpha': 3.43877999288044e-05, 'penalty': 'l2', 'max_iter': 1427}. Best is trial 0 with value: 0.3160970308095792.


Best SVM lineal: {'alpha': 0.008203850788364248, 'penalty': 'elasticnet', 'max_iter': 810}


In [12]:
# 12. Definir pipelines finales con parámetros tuneados

best_xgb = XGBClassifier(**study_xgb.best_params, objective='multi:softprob', eval_metric='mlogloss', tree_method='hist', random_state=42, use_label_encoder=False)
xgb_pipeline = make_base_pipeline(best_xgb)

best_svm_linear = SGDClassifier(loss="hinge", class_weight="balanced", random_state=42, **study_svm_linear.best_params)
svm_linear_pipeline = make_base_pipeline(best_svm_linear)

print('Pipelines finales listos.')


Pipelines finales listos.


In [13]:
# 13. Entrenar, evaluar y guardar resultados
modelos_finales = {'XGBoost': xgb_pipeline, 'SVM': svm_linear_pipeline}

resultados = []
mejores_modelos = {}

for nombre, pipe in modelos_finales.items():
    print('\\n==============================')
    print(f'🚀 Entrenando modelo: {nombre}')
    print('==============================')
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_valid)
    acc = accuracy_score(y_valid, y_pred)
    print(f'Accuracy {nombre}: {acc:.4f}')
    print('\\nClassification report:')
    print(classification_report(y_valid, y_pred))
    resultados.append({'Modelo': nombre, 'Accuracy': acc})
    mejores_modelos[nombre] = pipe
    dump(pipe, f'modelo_entrenado_{nombre}.joblib')

df_resultados = pd.DataFrame(resultados).sort_values('Accuracy', ascending=False)
display(df_resultados)


\n==============================
🚀 Entrenando modelo: XGBoost


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [21:05:17] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy XGBoost: 0.4318
\nClassification report:
              precision    recall  f1-score   support

           0       0.46      0.57      0.50     51896
           1       0.33      0.28      0.30     51682
           2       0.33      0.25      0.29     51486
           3       0.55      0.63      0.58     52686

    accuracy                           0.43    207750
   macro avg       0.42      0.43      0.42    207750
weighted avg       0.42      0.43      0.42    207750

\n==============================
🚀 Entrenando modelo: SVM
Accuracy SVM: 0.2536
\nClassification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     51896
           1       0.00      0.00      0.00     51682
           2       0.00      0.00      0.00     51486
           3       0.25      1.00      0.40     52686

    accuracy                           0.25    207750
   macro avg       0.06      0.25      0.10    207750
weighted avg       0.06      0.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Modelo,Accuracy
0,XGBoost,0.431759
1,SVM,0.253603
